In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount',
             'denial_rate',
             'avg_approved_loan_size'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data = data[data["state"] == 4]

In [5]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100

In [6]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount,denial_rate,avg_approved_loan_size
which_bank,,,,,,
All Other Banks,68.565726,81.572608,1.900577e+07,2.199419e+07,0.832347,274955.415563
Bank of West,1.530973,2.017699,3.911652e+05,4.848673e+05,0.877777,276262.929016


In [8]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications']

In [9]:
Quintiles_loan_table3.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                               
All Other Banks 20                                        0.607502   
                40                                        0.340795   
                60                                        0.049575   
                80                                        0.002088   
                100                                       0.000040   
Bank of West    20                                        0.660886   
                40                                        0.273603   
                60                                        0.050096   
                80                                        0.015414   
                100                                       0.000000   

                                        $loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                                
All Other Banks 20                                         0.644903   
                40                                         0.314177   
                60                                         0.039376   
                80                                         0.001518   
                100                                        0.000027   
Bank of West    20                                         0.702047   
                40                                         0.256702   
                60                                         0.031824   
                80                                         0.009426   
                100                                        0.000000   

                                        loan_approval_rate  
which_bank      pct_minority_Quintiles                      
All Other Banks 20                                0.846464  
                40                                0.835310  
                60                                0.808082  
                80                                0.796935  
                100                               0.800000  
Bank of West    20                                0.744035  
                40                                0.830409  
                60                                0.787879  
                80                                0.421053  
                100                                    NaN

In [10]:
data['pct_White'] = data['Tot.WhitePop']/data['Tot.Pop'] * 100
data['pct_Black'] = data['Tot.BlackPop ']/data['Tot.Pop'] * 100
data['pct_AmericIndian'] = data['Tot.AmericIndianPop']/data['Tot.Pop'] * 100
data['pct_Asian'] = data['Tot.AsianPop']/data['Tot.Pop'] * 100
data['pct_NativeHawaiianPacific'] = data['Tot.NativeHawaiianPacificPop']/data['Tot.Pop'] * 100
data['pct_OtherRaceAlone'] = data['Tot.OtherRaceAlonePop']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace'] = data['Tot.TwoOrMoreRace']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace(Some other race)'] = data['Tot.TwoOrMoreRace(Some other race)']/data['Tot.Pop'] * 100
data['pct_Hispanic/Latino'] = data['Tot.Hispanic/Latino']/data['Tot.Pop'] * 100
data['pct_NotHispanic/Latino'] = data['Tot.NotHispanic/Latino']/data['Tot.Pop'] * 100

In [11]:
data['major_minority'] = data[['pct_Black', 'pct_AmericIndian', 'pct_Asian','pct_NativeHawaiianPacific',
                                        'pct_OtherRaceAlone','pct_TwoOrMoreRace','pct_TwoOrMoreRace(Some other race)',
                                        'pct_Hispanic/Latino']].idxmax(axis=1)
data['major_minority'] = data['major_minority'].apply(lambda x: 'Black' if x=='pct_Black' 
                                                                        else 'American Indian' if x=='pct_AmericIndian' 
                                                                        else 'Asian' if x=='pct_Asian' 
                                                                        else 'Native Hawaiian Pacific' if x=='pct_NativeHawaiianPacific' 
                                                                        else 'Other Race Alone' if x=='pct_OtherRaceAlone' 
                                                                        else 'Two Or More Race' if x=='pct_TwoOrMoreRace' 
                                                                        else 'Two Or More Race(Some other race)' if x=='pct_TwoOrMoreRace(Some other race)' 
                                                                        else 'Hispanic/Latino')

In [12]:
Quintiles_loan_table4 = data.groupby(['which_bank', "major_minority"])[hmda_vars].sum()
Quintiles_loan_table4['loan_pct_in_minority_group'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['$loan_pct_in_minority_group'] = Quintiles_loan_table4['sum_approved_loans']/Quintiles_loan_table4['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['loan_approval_rate'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_applications']
Quintiles_loan_table4.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      major_minority                                 
All Other Banks American Indian                     0.005119   
                Asian                               0.049877   
                Black                               0.013952   
                Hispanic/Latino                     0.922490   
                Two Or More Race                    0.008562   
Bank of West    American Indian                     0.038536   
                Asian                               0.055877   
                Black                               0.019268   
                Hispanic/Latino                     0.878613   
                Two Or More Race                    0.007707   

                                  $loan_pct_in_minority_group  \
which_bank      major_minority                                  
All Other Banks American Indian                      0.004520   
                Asian                                0.071203   
                Black                                0.014627   
                Hispanic/Latino                      0.898167   
                Two Or More Race                     0.011484   
Bank of West    American Indian                      0.030542   
                Asian                                0.085027   
                Black                                0.016364   
                Hispanic/Latino                      0.860526   
                Two Or More Race                     0.007541   

                                  loan_approval_rate  
which_bank      major_minority                        
All Other Banks American Indian             0.817308  
                Asian                       0.868858  
                Black                       0.850673  
                Hispanic/Latino             0.838754  
                Two Or More Race            0.873975  
Bank of West    American Indian             0.555556  
                Asian                       0.852941  
                Black                       0.769231  
                Hispanic/Latino             0.763819  
                Two Or More Race            1.000000